# Convection-Diffusion

A fundamental test for the correct implementation of diffusive and
convective transport of a scalar quantity.
This validation test follows "H K Versteeg and W Malalasekera, An Introduction to Computational Fluid Dynamics
THE FINITE VOLUME METHOD, Second Edition, 2007" examples 5.1



In [7]:
%reset -f
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../src/MultiphaseTestBench')
import Manager as mptb


# the temperature along the x-axis should develop as:
def yFunc(x, case):
    if case == 1:
        return (2.7183-np.exp(x))/1.7183
    else:
        return 1+(1-np.exp(25*x))/(7.2e10)

case = 1   # 1,2,3

# geometric parameters
LenX = 1
LenY = 0.3
resolution = 20 if case == 3 else 5

#velocity in x direction: [ m/s ]
velX = 0.1 if case == 1 else 2.5

# thermal conductivity [ W / (m K) ]
k = 0.1

# const temperature at left/right boundaries [K]
T_l = 1
T_r = 0

geom = mptb.createGeometry( 'rectangle', [LenX, LenY] )
mesh = mptb.createMesh( geom, res=resolution )

# example: incompressible flow but with temp-dep density, i.e. BoussinesqFlow
# mulitphase stokes flow
myFlowModels = {
    'T' : mptb.TransportModels.scalarTransport
}

passiveFields = {
    'u' : 'faces_u',
    'v' : 'faces_v'
}

mptb.initialize(myFlowModels, mesh, geom, passiveFields=passiveFields )

T = mptb.getField('T')
T.govModel.setDiffusionCoefficient(value=k)

u = mptb.getField('u')
u.fill(velX)

# changing boundary conditions
mptb.defineBoundaryCondition(field=T, boundaryName='top', type='zeroGradient')
mptb.defineBoundaryCondition(field=T, boundaryName='bottom', type='zeroGradient')
mptb.defineBoundaryCondition(field=T, boundaryName='left', type='fixedValue', value=T_l )
mptb.defineBoundaryCondition(field=T, boundaryName='right', type='fixedValue', value=T_r )

#mptb.updateSource(field=T, value=0.0, mesh=mesh )

T.data = mptb.solve(T)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
# post-processing:
import numpy as np
import matplotlib.pyplot as plt

mptb.display(T, mesh)

nbcellsX = mesh.cells_x
cellSpacing = mesh.uniformSpacing
L = geom.lenX

xSim = np.linspace(0+0.5*cellSpacing,LenX-0.5*cellSpacing,nbcellsX)
ySim = T.data[0,:]

xTheo = np.linspace(0,LenX, 100)
yTheor = np.array([yFunc(x, case) for x in xTheo])

ax = plt.gca()
ax.plot(xSim, ySim, 'x', label='simulation')
ax.plot(xTheo, yTheor, label='theoretical')

plt.legend()
plt.show()

print(ySim)

In [ ]:
# shutting the interpreter down, so I can a fresh instance next time.
# my modules are automatically checked for updates
import os
os._exit(00)